# Alignment

In [3]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO, AlignIO
import pandas as pd

from katlas.utils import *
from katlas.data import *

## Prepare fasta file for clustalo

In [4]:
# kd=pd.read_excel('out/uniprot_kd.xlsx')

In [5]:
kd=Data.get_kd_uniprot()

In [6]:
kd.columns

Index(['kd_ID', 'Uniprot', 'Entry Name', 'Protein names', 'Gene Names',
       'Gene Names (primary)', 'Organism', 'kd_note', 'kd_evidence',
       'kd_start', 'kd_end', 'kd_seq', 'Domain [FT]', 'Domain [CC]', 'Region',
       'Motif', 'Protein families', 'Reactome', 'ComplexPortal',
       'Subcellular location [CC]', 'Gene Ontology (biological process)',
       'Tissue specificity', 'Interacts with', 'Subunit structure',
       'Function [CC]', 'Activity regulation', 'full_seq', 'D1', 'D2', 'D3',
       'N1', 'active_D1_D2'],
      dtype='object')

In [17]:
get_fasta(kd,
          seq_col='kd_seq',
          id_col='kd_ID',
          path='raw/kinase_domains.fasta')

5536


For human only:

In [ ]:
# human = kd[kd.Organism=='Homo sapiens (Human)']

# get_fasta(human,path="raw/human_kinase_domains.fasta")

for PI3K catalytic domain

In [ ]:
# pi3k = kd[kd.kd_note=='PI3K/PI4K catalytic']

# get_fasta(pi3k,path="raw/pi3k_kinase_domains.fasta")

## Run clustalo

```bash
sudo apt-get update
sudo apt-get install clustalo
clustalo -i kinase_domains.fasta -o kinase_domains.aln --force --outfmt=clu
```

In [ ]:
# run_clustalo(input_fasta='raw/kinase_domains.fasta', 
#              output_aln = "raw/kinase_domains.aln")

## Analyze alignment

In [12]:
df = aln2df("raw/kinase_domains.aln")

In [13]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,3425,3426,3427,3428,3429,3430,3431,3432,3433,3434
A0A075F7E9_LERK1_ORYSI_KD1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
A0A078BQP2_GCY25_CAEEL_KD1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
A0A078CGE6_M3KE1_BRANA_KD1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
A0A0G2K344_PK3CA_RAT_KD1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
A0A0H2ZM62_HK06_STRP2_KD1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [14]:
# change type to str to save in parquet
df.columns = df.columns.astype(str)

In [16]:
df.to_parquet('raw/uniprot_kd_align.parquet')

In [11]:
# reverse back
df.columns = df.columns.astype(int)

## Frequency of aa at position 1-3434

In [35]:
# Compute residue frequency at each position
counts_df = df.apply(lambda col: col.value_counts(), axis=0).fillna(0)
freq_df = counts_df.div(counts_df.sum(axis=0), axis=1)

In [37]:
freq_df.head()

,1,2,3,4,5,6,7,8,9,10,...,3425,3426,3427,3428,3429,3430,3431,3432,3433,3434
-,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,...,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819,0.999819
A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D,0.000000,0.000181,0.000000,0.000000,0.000181,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000181,0.000181,0.000000,0.000000,0.000000,0.000181,0.000000,0.000000,0.000000
E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000181,0.000000,0.000000


In [38]:
# remove '-' first line
max_series=freq_df.iloc[1:,:]

In [39]:
freq_max = pd.concat([max_series.idxmax(),max_series.max()],axis=1)

In [40]:
freq_max.columns = ['aa','max_value']

In [41]:
freq_max = freq_max.sort_values('max_value',ascending=False).reset_index(names='position')

In [42]:
out = freq_max[freq_max.max_value>0.1]

In [43]:
out

,position,aa,max_value
0,1549,N,0.815390
1,2618,D,0.809429
2,1724,D,0.800759
3,1525,D,0.791004
4,1730,G,0.775470
...,...,...,...
214,640,G,0.101879
215,193,E,0.101879
216,922,L,0.101337
217,603,R,0.101156


In [ ]:
# out.to_csv('out/align_freq_max_aa.csv',index=False)

## Locate active motif & save

- 1549 N is after HRD motif
- 2618 D is around D[IV]WS motif
- 1724 D is DFG motif
- 1525 D is HRD motif

In [65]:
df=df.reset_index()

In [67]:
kd['D1']=(df[1525]=='D').astype(int) # HRD

In [68]:
kd['D2'] = (df[1724]=='D').astype(int) #DFG

In [70]:
kd['D3']=(df[2618]=='D').astype(int)

In [71]:
kd['N1'] = (df[1549]=='N').astype(int)

In [72]:
active_col = ['D1','D2']

In [73]:
kd['active_D1_D2'] = (kd[active_col].sum(1)==len(active_col)).astype(int)

In [74]:
kd['active_D1_D2'].value_counts()

active_D1_D2
1    4209
0    1327
Name: count, dtype: int64

In [75]:
active_all = kd[kd.active_D1_D2==1]

In [76]:
active_all.shape

(4209, 32)

In [ ]:
# active_all.to_excel('out/uniprot_kd_active_D1_D2.xlsx',index=False)

In [ ]:
# kd.to_excel('out/uniprot_kd_motif_labeled.xlsx',index=False)

Take a look of their identity:

In [77]:
kd[kd.active_D1_D2==1].kd_note.value_counts()

kd_note
Protein kinase               4005
PI3K/PI4K catalytic            71
Protein kinase 2               61
Protein kinase 1               37
Histidine kinase               34
Alpha-type protein kinase       1
Name: count, dtype: int64

In [78]:
kd[kd.active_D1_D2==0].kd_note.value_counts()

kd_note
Histidine kinase                  591
Protein kinase                    525
PI3K/PI4K catalytic                97
Protein kinase 1                   36
Alpha-type protein kinase          21
Protein kinase; inactive           13
Protein kinase 2                   12
HWE histidine kinase domain        11
Guanylate kinase-like               5
Histidine kinase 2                  4
Histidine kinase 1                  4
Amino-acid kinase domain (AAK)      3
Kinase domain                       2
Histidine kinase; first part        1
Histidine kinase; second part       1
Protein kinase; truncated           1
Name: count, dtype: int64

## Add KD info to human kinase info

In [79]:
kd = pd.read_excel('out/uniprot_kd_motif_labeled.xlsx')

In [80]:
human= kd[kd.Organism=='Homo sapiens (Human)']

In [81]:
kd_id_uniprot_map = human.groupby('Uniprot').agg({'kd_ID': lambda x: ','.join(x.unique()),
                                                  'active_D1_D2': lambda x: x,
                                                 })

In [82]:
kd_id_uniprot_map

,kd_ID,active_D1_D2
Uniprot,,
A4D2B8,A4D2B8_PM2P1_HUMAN_KD1,0
A4QPH2,A4QPH2_PI4P2_HUMAN_KD1,0
O00141,O00141_SGK1_HUMAN_KD1,1
O00238,O00238_BMR1B_HUMAN_KD1,1
O00311,O00311_CDC7_HUMAN_KD1,1
...,...,...
Q9Y616,Q9Y616_IRAK3_HUMAN_KD1,0
Q9Y6E0,Q9Y6E0_STK24_HUMAN_KD1,1
Q9Y6M4,Q9Y6M4_KC1G3_HUMAN_KD1,1


In [ ]:
# kd_id_uniprot_map.to_csv('raw/kd_ID_uniprot_map.csv')

Get the csv merge with kinase info. Manually match KD1 and KD2 to the kinase info by considering active_D1_D2.

The data is updated on github.

In [83]:
from katlas.data import *

In [88]:
kd_info =Data.get_kd_uniprot()

In [90]:
kd_info.head()

,kd_ID,Uniprot,Entry Name,Protein names,Gene Names,Gene Names (primary),Organism,kd_note,kd_evidence,kd_start,...,Interacts with,Subunit structure,Function [CC],Activity regulation,full_seq,D1,D2,D3,N1,active_D1_D2
0,A0A075F7E9_LERK1_ORYSI_KD1,A0A075F7E9,LERK1_ORYSI,G-type lectin S-receptor-like serine/threonine...,LECRK1 LECRK OsI_14840,LECRK1,Oryza sativa subsp. indica (Rice),Protein kinase,ECO:0000255|PROSITE-ProRule:PRU00159,523,...,None,SUBUNIT: Interacts (via kinase domain) with AD...,FUNCTION: Involved in innate immunity. Require...,None,MVALLLFPMLLQLLSPTCAQTQKNITLGSTLAPQGPASSWLSPSGD...,1,1,1,1,1
1,A0A078BQP2_GCY25_CAEEL_KD1,A0A078BQP2,GCY25_CAEEL,Receptor-type guanylate cyclase gcy-25 (EC 4.6...,gcy-25 Y105C5B.2,gcy-25,Caenorhabditis elegans,Protein kinase,ECO:0000255|PROSITE-ProRule:PRU00159,464,...,None,None,FUNCTION: Guanylate cyclase involved in the pr...,None,MLLLLLLLKISTFVDSFQIGHLEFENSNETRILEICMKNAGSWRDH...,0,0,1,0,0
2,A0A078CGE6_M3KE1_BRANA_KD1,A0A078CGE6,M3KE1_BRANA,MAP3K epsilon protein kinase 1 (BnM3KE1) (EC 2...,M3KE1 BnaA03g30290D GSBRNA2T00111755001,M3KE1,Brassica napus (Rape),Protein kinase,ECO:0000255|PROSITE-ProRule:PRU00159,20,...,None,None,FUNCTION: Serine/threonine-protein kinase invo...,None,MARQMTSSQFHKSKTLDNKYMLGDEIGKGAYGRVYIGLDLENGDFV...,1,1,1,1,1
3,A0A0G2K344_PK3CA_RAT_KD1,A0A0G2K344,PK3CA_RAT,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",Pik3ca,Pik3ca,Rattus norvegicus (Rat),PI3K/PI4K catalytic,ECO:0000255|PROSITE-ProRule:PRU00269,765,...,None,SUBUNIT: Heterodimer of a catalytic subunit PI...,FUNCTION: Phosphoinositide-3-kinase (PI3K) pho...,None,MPPRPSSGELWGIHLMPPRILVECLLPNGMIVTLECLREATLVTIK...,0,0,0,0,0
4,A0A0H2ZM62_HK06_STRP2_KD1,A0A0H2ZM62,HK06_STRP2,Sensor histidine protein kinase HK06 (EC 2.7.1...,hk06 SPD_2019,hk06,Streptococcus pneumoniae serotype 2 (strain D3...,Histidine kinase,ECO:0000255|PROSITE-ProRule:PRU00107,239,...,None,None,FUNCTION: Member of the two-component regulato...,None,MIKNPKLLTKSFLRSFAILGGVGLVIHIAIYLTFPFYYIQLEGEKF...,0,0,0,0,0


In [91]:
kd_info.shape

(5536, 32)

In [84]:
info=Data.get_kinase_info()

In [86]:
info.columns

Index(['kinase', 'ID_coral', 'uniprot', 'gene', 'modi_group', 'group',
       'family', 'subfamily_coral', 'subfamily', 'in_pspa_st', 'in_pspa_tyr',
       'in_pspa', 'in_cddm', 'kd_ID', 'active_D1_D2', 'active_kd_ID',
       'pspa_ID', 'pseudo', 'pspa_category_small', 'pspa_category_big',
       'cddm_big', 'cddm_small', 'length', 'human_uniprot_sequence',
       'kinasecom_domain', 'nucleus', 'cytosol', 'cytoskeleton',
       'plasma membrane', 'mitochondrion', 'Golgi apparatus',
       'endoplasmic reticulum', 'vesicle', 'centrosome', 'aggresome',
       'main_location'],
      dtype='object')

In [87]:
info.kd_ID

0       Q2M2I8_AAK1_HUMAN_KD1
1      Q6ZMQ8_LMTK1_HUMAN_KD1
2       P00519_ABL1_HUMAN_KD1
3       P42684_ABL2_HUMAN_KD1
4      Q8NER5_ACV1C_HUMAN_KD1
                ...          
518      P07947_YES_HUMAN_KD1
519    O00506_STK25_HUMAN_KD1
520    Q56UN5_M3K19_HUMAN_KD1
521    Q9NYL2_M3K20_HUMAN_KD1
522    P43403_ZAP70_HUMAN_KD1
Name: kd_ID, Length: 523, dtype: object